# Rates for Univariate Sobolev Regression

Why would we prefer a model with a bounded second derivative over one with a bounded first derivative? For example,
the $p=2$ version of this Sobolev model below over the $p=1$ version?

$$
\begin{aligned}
\mathcal{M}^p 
&= \left\{ m : \lVert m^{(p)} \rVert_{L_2} \le B \right\} \\
&=\left\{ \sum_{j = 0}^\infty m_j \phi_j : \sum_{j=0}^\infty \lambda_j^p m_j^2 \le B^2 \right\} \\
& \text{ for } \phi_{j}(x) = \sqrt{2}\cos(\pi j x) \ \text{ and } \ \lambda_{j}=\pi^2 j^2. 
\end{aligned}
$$

It overfits less. Let's compare the $p=1$ and $p=2$ models fit the data with the signal 
$\mu(x)=\sqrt{2}\cos(\pi x)$. Note that $\mu=\phi_1$, so it's in the $p=1$ model for $B \ge \lambda_1=\pi^2$
and the $p=2$ model for $B \ge \lambda_1^{2}=\pi^4$. We'll use those values for $B$ below.

You'll have to run a few code blocks, since we haven't yet loaded any packages or implemented sobolev regression.
The next two blocks are copied from the sobolev regression lab.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
    CVXR::add_to_solver_blacklist('OSQP')  
})

lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    	  panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				  panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	              panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		          axis.ticks.x = element_blank(),
		          axis.ticks.y = element_blank(),
		          axis.text.x  = element_text(colour = "#aaaaaa"),
		          axis.text.y  = element_text(colour = "#aaaaaa"),
		          axis.title.x  = element_text(colour = "#aaaaaa"),
		          axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)

In [ ]:

sobolevreg = function(X,Y, p=1, B=1, epsilon=.001){
  N = ceiling((1/pi)*(B/epsilon)^(1/p))
  Phi = outer(X, 0:(N-1), function(x,j) cos(pi*j*x)) 
  lambda = pi^2 * (0:(N-1))^(2*p)
  # specify the parameters and constraints.
  b = Variable(N)
  m = Phi %*% b
  mse = sum((Y - m)^2)/length(Y) 
  constraints = list(sum(lambda * b^2) <= B^2)

  # solve and extract the solution
  solved = solve(Problem(Minimize(mse), constraints))
  beta.hat = solved$getValue(b)
  
  # package up the results in a model object and return it
  model = list(beta.hat=beta.hat, p=p, input=list(X=X, Y=Y)) 
  attr(model, "class") = "sobolevreg"
  model
}

predict.sobolevreg = function(model, newdata=model$input) { 
  x = newdata$X
  beta.hat = model$beta.hat
  N = length(beta.hat)
  Phi.x = outer(x, 0:(N-1), function(x,j) cos(pi*j*x)) 
  
  Phi.x %*% beta.hat 
}

In [ ]:
n = 50
sigma = .2  

mu = function(x) { sqrt(2)*cos(pi*x) } 
X = runif(n)
Y = mu(X) + sigma*rnorm(length(X))

fit.model.1 = \(X,Y)sobolevreg(X,Y,p=1,B=pi^2)
fit.model.2 = \(X,Y)sobolevreg(X,Y,p=2,B=pi^4)

x = seq(0,1,by=.01)
newdata = data.frame(X=x)
ggplot() + 
  geom_line(aes(x=x, y=mu(x))) + 
  geom_point(aes(x=X,y=Y), alpha=.1) + 
  geom_line(aes(x=x, y=predict(fit.model.1(X,Y), newdata)), color="blue") + 
  geom_line(aes(x=x, y=predict(fit.model.2(X,Y), newdata)), color="red") 

Even though we've chosen $B$ for each model so that $\mu$ is just barely in it, i.e. we've chosen $B_1$
and $B_2$ so that $\lVert\mu' \rVert_{L_2}  = B_1$ and $\lVert\mu'' \rVert_{L_2} = B_2$, the $p=2$ model is still
giving us a better fit. The overall shape is the same, and it's not perfect, but it jumps around a lot less. 

To get a quantitative sense of the difference, let's vary sample size, plot some error curves, and estimate rates of convergence. We have code for that. It does take a few minutes to run.

In [ ]:
ns = c(50,100,200,400,800)

models  = list(p.is.1=fit.model.1,
               p.is.2=fit.model.2)

tabulate.errors.for.sample = function(X,Y,models,ns) { 
  ns |> map(function(n) {
    models |> map(function(model.fit) {
      model = model.fit(X[1:n], Y[1:n])
      muhat = function(x) { predict(model, newdata=data.frame(X=x)) }
      error = sqrt(mean((mu(X[1:n]) - muhat(X[1:n]))^2))
      data.frame(n=n, error=error)
    }) |> list_rbind(names_to='model')
  }) |> list_rbind()
}
tabulate.errors = function(replications=10) { 
  1:replications |> map(function(rep) {
    X           = runif(max(ns))
    epsilon     = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    tabulate.errors.for.sample(X, Y, models, ns)
  }) |> list_rbind(names_to='rep')
}

tab = tabulate.errors(replications=25)

Here are the error curves.

In [ ]:
error.curves = ggplot(tab, aes(x=n, y=error, color=model)) + 
    stat_summary(geom='line', fun=mean) +  
    stat_summary(geom='pointrange', fun.data=mean_se,
    		 position=position_dodge(5)) 

error.curves

Here are the rates for each model. They're $n^{-\beta}$ where $\beta$ is in the table below.

In [ ]:
error.model = function(group) { 
	reg.data = group |> group_by(n) |> summarize(error=mean(error))
	nls(formula = error ~ a*n^(-b), data=reg.data, 
	    start=list(a=1, b=1/2))
}

rates = tab |> 
  group_by(model) |>
	group_modify(function(group,...) {
	    rate.model = error.model(group)
		  data.frame(alpha=coef(rate.model)[1], beta=coef(rate.model)[2])
	})

rates

And here, as evidence that the rates are meaningful, are the error curves this $\alpha n^{-\beta}$ model predicts overlaid on the actual error curves.

In [ ]:
error.predictions = tab |> group_by(model) |>
	group_modify(function(group,...) {
		rate.model = error.model(group)
		data.frame(n=ns, error = predict(rate.model, newdata=data.frame(n=ns)))
	})

error.curves + 
    geom_line(aes(x=n, y=error, color=model), alpha=.4, linewidth=2, data=error.predictions)

# Rates for Bivariate Sobolev Regression

A cube-root rate isn't bad, so maybe we don't need the $p=2$ model to get a good fit when $X$ is one-dimensional. 
Let's check out the bivariate case using the *isotropic* Sobolev model.

$$
\begin{aligned}
\mathcal{M}^p 
&= \left\{ m \ : \ \left\langle -\Delta^p m, \ m \right\rangle_{L_2} \le B \right\} \\
&= \left\{ \sum_{j \in \mathbb{N}^2} m_j \phi_j  \ : \sum_{j \in \mathbb{N}^2} \lambda_j^p \ m_j^2  \le B^2 \right\} 
&&\text{ for } \quad \phi_j(x) =  \cos(\pi j_1 x_1) \cos(\pi j_2 x_2) \\ 
&&&\text{ and } \quad \lambda_j = \pi^2 \lVert j \rVert_2^2
\end{aligned}
$$

We'll use this model to fit noisy image data. To make the computation manageable, we'll use pretty small images.
We'll make them $10 \times 10$, $14\times14$, and $20\times20$ giving us sample sizes (number of pixes) of $100$, $196$, and $400$ respectively. We'll use two different signals:

- A smooth bump: the bivariate normal density function.
- A sharp checkerboard pattern.

Here are what these images look like with and without added noise.
We'll use the `gridExtra` package to plot the images side by side. You may need to install it.

In [ ]:
suppressPackageStartupMessages({
    library(gridExtra)
})

In [ ]:
image.from.function = function(f, noise.level=.2, px=10){
  grid = expand.grid(x1=seq(0, 1, length.out = px), 
                     x2=seq(0, 1, length.out = px))
  eps = noise.level*matrix(runif(px*px, -1, 1), nrow=px)  
  matrix(1 + f(grid$x1, grid$x2) + eps, nrow=px, ncol=px)
}
bump = function(noise.level=.2, px=10) {
  f = function(x1,x2) { dnorm(3*sqrt((x1-0.5)^2 + (x2-0.5)^2)) }
  f |> image.from.function(noise.level, px)
}
checkerboard = function(noise.level=.2, px=10){
  f = function(x1,x2) { as.numeric((x1 <= .5) == (x2 <= .5)) }
  f |> image.from.function(noise.level, px)
}

image.to.df = function(image) {
  grid = expand.grid(x1=seq(0, 1, length.out = nrow(image)), 
                     x2=seq(0, 1, length.out = ncol(image)))
  grid$Y = as.vector(image)
  grid
}
image.from.df = function(df, dim=sqrt(nrow(df))*c(1,1)) {
  matrix(df$Y, nrow=dim[1], ncol=dim[2])
}

show.image = function(image) {
  df = image.to.df(image)
  ggplot(df, aes(x = x1, y = x2, fill = Y)) +
    geom_raster() +
    scale_fill_gradient(low = "black", high = "white") +
    coord_equal() + guides(fill='none')
}

multires.plot = function(im) { 
  grid.arrange(im(noise.level=0,  px=10) |> show.image(),
               im(noise.level=.2, px=10) |> show.image(),
               im(noise.level=0,  px=14) |> show.image(),
               im(noise.level=.2, px=14) |> show.image(),
               im(noise.level=0,  px=20) |> show.image(),
               im(noise.level=.2, px=20) |> show.image(),
               ncol=2)
}
multires.plot(bump)
multires.plot(checkerboard)

And here's some code to fit our Sobolev model to these images. Once we've got our basis functions $\phi_j$ and the corresponding eigenvalues $\lambda_j$, the code is the same as in the univariate case. 

In [ ]:
cosine.basis.2d = function(p, B=1, epsilon=.1) {
  N.1 = ceiling((B / epsilon)^(1/p) / pi) 
  frequencies   =  expand.grid(j1=0:N.1, j2=0:N.1)
  j = cbind(frequencies$j1, frequencies$j2)
  lambda = (pi^2 * (j[,1]^2 + j[,2]^2) )^p
  Phi = function(data) {
    phi = matrix(nrow=nrow(data), ncol=nrow(j))
    for(ij in 1:nrow(j)) {
	    phi[,ij] = 2*cos(pi*j[ij,1]*data$x1)*cos(pi*j[ij,2]*data$x2)
    }
    phi
  }
  list(lambda=lambda, Phi=Phi)
}

sobolevreg2d = function(data, p=1, epsilon=.1, B=1,
                        basis=cosine.basis.2d(p, B, epsilon)) {
  lambda = basis$lambda
  Phi = basis$Phi(data)
  Y = data$Y

  # specify the parameters and constraints.
  b = Variable(length(lambda))
  m = Phi %*% b
  mse = sum((Y - m)^2)/length(Y) 
  constraints = list(sum(lambda * b^2) <= B^2)

  # solve and extract the solution
  solved = solve(Problem(Minimize(mse), constraints))
  beta.hat = solved$getValue(b)
  
  # package up the results in a model object and return it
  model = list(beta.hat=beta.hat, basis=basis, input=data) 
  attr(model, "class") = "sobolevreg2d"
  model
}

predict.sobolevreg2d = function(model, newdata=model$input) { 
  Phi.x = model$basis$Phi(newdata)
  Phi.x %*% model$beta.hat 
}

Below we'll see what happens when we denoise the bump image using the $p=1$ and $p=2$ Sobolev models. In the plot below:

  - The top left image is the signal $\mu$
  - The top right image is the image $\mu + \varepsilon$ that we're denoising.
  - The bottom left image is the denoised image using the $p=1$ model.
  - The bottom right image is the denoised image using the $p=2$ model.

To get a fair comparison, we'll use cross-validation to choose the regularization parameter $B$ for each model.
Both do pretty well, which is interesting given how little the noisy image looks like the signal to the naked eye.
But the $p=2$ model does a bit better. 

In [ ]:
set.seed(0)
px = 20
im = bump
noisy.image = im(noise.level=.2, px=px)
clean.image = im(noise.level=0, px=px) 

denoise.image = function(image, p=1, B=seq(.25,10,by=.25)) {
  denoised = image.to.df(image)
  
  if(length(B) > 1) {
    cv.errors = map_dbl(B, function(b) {
      train = sample(1:nrow(denoised)) <= nrow(denoised)/2
      model = sobolevreg2d(denoised[train,], p=p, B=b) 
      prediction = predict(model, denoised[!train,])
      mean((prediction - denoised$Y[!train])^2)
    })
    best.B = B[which.min(cv.errors)]
  } else {
    best.B = B
  }
  model = sobolevreg2d(denoised, p=p, B=best.B) 
  denoised$Y = predict(model)
  im = image.from.df(denoised)
  
  attr(im, 'B') = best.B
  im
}


denoised.1 = noisy.image |> denoise.image(p=1)
denoised.2 = noisy.image |> denoise.image(p=2) 

c(mean((denoised.1-clean.image)^2), mean((denoised.2-clean.image)^2))

grid.arrange(clean.image |> show.image(),
             noisy.image |> show.image(), 
             denoised.1 |> show.image(), 
             denoised.2 |> show.image(), 
             ncol=2)

But that's just one signal. If you swap out the bump for the checkerboard, which is much less smooth, you'll see the $p=1$ model does better. To do that, change line 3 above from `im=bump` to `im=checkerboard`. And it's just one random draw from the distribution of noisy images with that signal. 

Let's plot the root-mean-square error of each estimator as a function of sample size, like we did in the univariate case. 

In [ ]:
set.seed(0)
pxs = c(10,14,20)
models  = list(p.is.1=\(im)denoise.image(im, p=1),
               p.is.2=\(im)denoise.image(im, p=2))
signals = list(bump=bump, checkerboard=checkerboard)

tabulate.errors.2d = function(replications=10) {
    pxs |> map(function(px) {
      signals |> map(function(im) { 
        clean.image = im(noise.level=0, px=px) 
        1:replications |> map(function(rep) {
          noisy.image = im(noise.level=.2, px=px)
          models |> map(function(denoise) {
            denoised.image = denoise(noisy.image)
            data.frame(n=px^2,
                       B=attr(denoised.image, 'B'), 
                       error=mean((denoised.image - clean.image)^2))
          }) |> list_rbind(names_to='model')
        }) |> list_rbind(names_to='rep')
      }) |> list_rbind(names_to='signal')
    }) |> list_rbind()
}

tab.2d = tabulate.errors.2d(replications=25)

What do you make of these curves?

In [ ]:
error.curves = ggplot(tab.2d, aes(x=n, y=error, color=model)) + 
    stat_summary(geom='line', fun=mean) +  
    stat_summary(geom='pointrange', fun.data=mean_se,
    		 position=position_dodge(5)) +
    geom_line(aes(group=interaction(rep,model)), linewidth=.05) +
    facet_wrap(.~signal, scales='free_y')
error.curves

Here's a plot of the average regularization parameter $B$ chosen by cross-validation.

Why are they increasing with sample sizefor the checkerboard signal?

In [ ]:
B.curves = ggplot(tab.2d, aes(x=n, y=B, color=model)) + 
    stat_summary(geom='line', fun=mean) +  
    stat_summary(geom='pointrange', fun.data=mean_se,
    		 position=position_dodge(5)) +
    geom_line(aes(group=interaction(rep,model)), linewidth=.05) +
    facet_wrap(.~signal)
B.curves